In [89]:
import numpy as np
# import pathogenprofiler as pp
from sklearn.mixture import GaussianMixture
from sklearn.metrics import mean_squared_error
# import fastq2matrix as fm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from scipy.stats import norm
import subprocess
from scipy.stats import kurtosis, skew
import re
import os


In [149]:
vcf_file = 'test_data/ERR6634978-ERR6635032-0100.vcf.gz'
json_file = 'test_data/ERR6634978-ERR6635032-2080.results.json'

In [130]:
VCF_FILE_PATH='/mnt/storage7/lwang/trial_tb_philippines/data/processed/seqtk/freebayesVCF'
JSON_FILE_PATH='/mnt/storage7/lwang/trial_tb_philippines/data/processed/seqtk/freebayesVCF/results'
NAME_FILE='/mnt/storage7/lwang/trial_tb_philippines/data/seqtk/sample_name.txt'

In [154]:
tb_profiler_predictions = []

def tb_profiler_pred(json_file):
    output = []
    json_results = json.load(open(json_file))
    for x in json_results['lineage']:
        if re.search("^lineage1.2.1$", x['lin']):
            output.append(x['frac'])
        if re.search("^lineage4.3.4$", x['lin']):
            output.append(x['frac'])
        
    if len(output) == 1:
        output.append(0)
    if output[0] < output[1]:
        temp = output[0]
        output[0] = output[1]
        output[1] = temp
    if output[0] > output[1]:
        cut_off =  output[1]/2
    else:
        cut_off =  output[-2]/2

    return output, cut_off


print(tb_profiler_pred(json_file))

([0.7554125219426565, 0.20933977455716588], 0.10466988727858294)


In [169]:
model_predictions = []

def model_pred(vcf_file, tail_cutoff=0.00):
    with open('mix_infection.csv', 'w') as f:
        subprocess.run("bcftools view -c 1 -m2 -M2 -T ^new_exclusion.bed %s | bcftools query -f '%%POS\\t%%REF\\t%%ALT[\\t%%GT\\t%%AD\\n]'" % vcf_file, shell=True, stdout=f, text=True)

#count how many column there is in the ROAO_proportion.csv file this is needed in order to read the csv in a a panda dataframe
    pos = []
    freqs = []
    with open('mix_infection.csv', 'r') as f:
        for l in f:
            row = l.strip().split()
            ads = [int(x) for x in row[4].split(",")]
            afs = [x/sum(ads) for x in ads]
            if afs[1]>1-tail_cutoff or afs[1]<tail_cutoff:
                continue
            pos.append(int(row[0]))
            freqs.append([afs[1]])
        # freqs = [[0.7],[0.6],[0.4]]    
        gm = GaussianMixture(n_components=2, random_state=0).fit(freqs)
        mu0 = gm.means_[1][0]
        mu1 = gm.means_[0][0]
    if mu0 > mu1:
        return [mu0, mu1]
    else:
        return [mu1, mu0]


print(model_pred(vcf_file))


[0.9976613585401825, 0.5891451515923425]


In [172]:
def frac_MSE(NAME_FILE=NAME_FILE, JSON_FILE_PATH=JSON_FILE_PATH, VCF_FILE_PATH=VCF_FILE_PATH):
    with open(NAME_FILE, 'r') as f:
        sample_list = [line.rstrip('\n') for line in f]

    sample_list_json=[]
    for i,x in enumerate(sample_list):
        x = list(x)
        x.append('.results.json')
        string = ''.join([letter for letter in x] )
        sample_list_json.append(string)
    

    sample_list_vcf=[]
    for i,x in enumerate(sample_list):
        x = list(x)
        x.append('.vcf.gz')
        string = ''.join([letter for letter in x] )
        sample_list_vcf.append(string)

    ratio = []
    for i in sample_list_json:
        temp = i.split('-')
        ratio.append(temp[2])
    
    ratio1=[]
    for i in ratio:
        temp = i.split('.')
        ratio1.append(temp[0])

    ratio = ratio1

    tb_profiler_predictions = []
    model_predictions = []

    for json, vcf, ratio in zip(sample_list_json, sample_list_vcf, ratio):
        JSON_FILE = os.path.join(JSON_FILE_PATH, json)
        tb_out_, cut_off = tb_profiler_pred(JSON_FILE)

        VCF_FILE = os.path.join(VCF_FILE_PATH, vcf)
        model_out_ = model_pred(vcf_file, tail_cutoff=cut_off)

        tb_profiler_predictions.extend(tb_out_)
        model_predictions.extend(model_out_)
    
    # print(tb_profiler_predictions)
    # print(model_predictions)

    MSE = mean_squared_error(tb_profiler_predictions, model_predictions, squared=False)

    return MSE

<h1>MSE

In [173]:
print(frac_MSE(NAME_FILE=NAME_FILE, JSON_FILE_PATH=JSON_FILE_PATH, VCF_FILE_PATH=VCF_FILE_PATH))

0.22378840741845601


In [31]:
model_pred(vcf_file, tail_cutoff=0.1)


[0.2352338544977916, 0.7659445772369651]


In [176]:
def plot_gm(gm,data):
    gm.covariances_[0][0][0]
    std0 = np.sqrt(gm.covariances_[0][0][0]) #(n_components, n_features, n_features)
    print('std0:',std0)
    mu0 = gm.means_[0][0]
    print(mu0)
    std1 = np.sqrt(gm.covariances_[1][0][0]) #component 1?
    print('std1:',std1)
    mu1 = gm.means_[1][0]
    print(mu1)
    x = np.linspace(0, 1, 100)
    p0 = norm.pdf(x, mu0, std0)
    p1 = norm.pdf(x, mu1, std1)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Histogram(x=[x[0] for x in data]),secondary_y=False)
    fig.add_trace(go.Scatter(x=x, y=p0, mode='lines'),secondary_y=True)
    fig.add_trace(go.Scatter(x=x, y=p1, mode='lines'),secondary_y=True)
    fig.show()

In [33]:
def plot_gaussian():
    pos = []
    freqs = []
    with open('mix_infection.csv', 'w') as f:
        subprocess.run("bcftools view -c 1 -m2 -M2 -T ^new_exclusion.bed %s | bcftools query -f '%%POS\\t%%REF\\t%%ALT[\\t%%GT\\t%%AD\\n]'" % vcf_file, shell=True, stdout=f, text=True)

#count how many column there is in the ROAO_proportion.csv file this is needed in order to read the csv in a a panda dataframe

    with open('mix_infection.csv', 'r') as f:
        for l in f:
            row = l.strip().split()
            ads = [int(x) for x in row[4].split(",")]
            afs = [x/sum(ads) for x in ads]
            pos.append(int(row[0]))
            freqs.append([afs[1]])

        mean_ = np.mean([x[0] for x in freqs])
        std_ = np.std([x[0] for x in freqs])
        print('mean:', mean_)
        print('std:', std_)
        print('2sd up', mean_-2*std_)
        print('2sd down', mean_+2*std_)

        print('skew:', skew([x[0] for x in freqs]))
        print('kurtosis:', kurtosis([x[0] for x in freqs]))


        fig = make_subplots()
        fig.add_trace(go.Histogram(x=[x[0] for x in freqs]),secondary_y=False)
        fig.add_vline(mean_-2*std_, line_dash='dash',line_color='red')
        fig.add_vline(mean_+2*std_, line_dash='dash',line_color='red')

        fig.show()



In [34]:
plot_gaussian()

mean: 0.6977412427292548
std: 0.25509024550504006
2sd up 0.18756075171917463
2sd down 1.2079217337393349
skew: -1.0221593974332557
kurtosis: -0.059565656853799887


In [35]:
def vcf_to_mix_model(vcf_file,plot=False,tail_cutoff=0.05,title="AF Histogram",return_freqs = False):
    pos = []
    freqs = []
    with open('mix_infection.csv', 'w') as f:
        subprocess.run("bcftools view -c 1 -m2 -M2 -T ^new_exclusion.bed %s | bcftools query -f '%%POS\\t%%REF\\t%%ALT[\\t%%GT\\t%%AD\\n]'" % vcf_file, shell=True, stdout=f, text=True)

#count how many column there is in the ROAO_proportion.csv file this is needed in order to read the csv in a a panda dataframe

    with open('mix_infection.csv', 'r') as f:
        for l in f:
            row = l.strip().split()
            ads = [int(x) for x in row[4].split(",")]
            afs = [x/sum(ads) for x in ads]
            if afs[1]>1-tail_cutoff or afs[1]<tail_cutoff:
                continue
            pos.append(int(row[0]))
            freqs.append([afs[1]])
        # freqs = [[0.7],[0.6],[0.4]]    
    gm = GaussianMixture(n_components=2, random_state=0).fit(freqs)
    if plot:
        plot_gm(gm,freqs)
    if return_freqs:
        return (gm,list(zip(pos,freqs)))
    else:
        return gm



In [36]:
def assign_variant_to_distrib(gm,freq,cutoff=0.95):
    probs = gm.predict_proba([[freq]])
    pred = gm.predict([[freq]])
    if probs[0][pred][0]>cutoff:
        return pred[0], probs[0][pred][0]
    else:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
        return None, pred[0], probs[0][pred][0]

In [177]:
gm = vcf_to_mix_model(vcf_file,plot=True,tail_cutoff=0.02)


std0: 0.18561016409299874
0.34791838929507174
std1: 0.011530977945801239
0.9678285446795938


In [241]:
json_results = json.load(open(json_file))
strain0 = []
strain1 = []
strainU = []

for var in json_results['dr_variants']:
    cluster = assign_variant_to_distrib(gm,var['freq'])
    if cluster[0] == 0:
        var['probs']= cluster
        strain0.append(var)
    elif cluster[0] == 1:
        var['probs']= cluster
        strain1.append(var)
    else:
        var['probs']= cluster[1:]
        strainU.append(var)

In [237]:
print([(v['gene'],v['change'],v['freq'],v['probs']) for v in strain0])
for x in strain0:
    for y in x['drugs']:
        print(y['drug'])

[('rpoB', 'p.His445Leu', 0.6901408450704225, (0, 0.9999998519095323)), ('rrs', 'n.514A>T', 0.8, (0, 0.9999999999875882)), ('fabG1', 'c.-15C>T', 0.7005347593582888, (0, 0.9999999482916717))]
rifampicin
streptomycin
isoniazid
ethionamide


In [238]:
print([(v['gene'],v['change'],v['freq'],v['probs']) for v in strain1])
for x in strain1:
    for y in x['drugs']:
        print(y['drug'])


[('rpoB', 'p.Ser450Leu', 0.32450331125827814, (1, 1.0)), ('katG', 'p.Ser315Thr', 0.24731182795698925, (1, 1.0)), ('embB', 'p.Met306Val', 0.19631901840490798, (1, 1.0))]
rifampicin
isoniazid
ethambutol


In [239]:
print([(v['gene'],v['change'],v['freq'],v['probs']) for v in strainU])
for x in strainU:
    for y in x['drugs']:
        print(y['drug'])

[]
